# Testing of Python classes with Jupyter Notebook

## Die Environment Variablen für AWS:
- AWS_ACCESS_KEY_ID
- AWS_SECRET_ACCESS_KEY

## Die Environment Variablen für AIR data query:
- AIR_QUALITY_API_TOKEN

### For local check make a copy of 0_config/credentialsTemplate and rename it to credentials (file ignore by git)

In [ ]:
basePath = "../"

In [ ]:
# Check the base directory
from os import listdir
from os.path import isfile, join
mypath = basePath
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

In [ ]:
import sys
import os
sys.path.insert(0,basePath+"./1_classes/")

import AirQuality
import S3

In [ ]:
from CredentialsManager import importCredentialsToEnvironment

importCredentialsToEnvironment(basePath)

## Test writing process

In [ ]:
reader = AirQuality.AirQualityReader(apiToken=os.environ['AIR_QUALITY_API_TOKEN'], basePath=basePath)
reader.readConfig('0_config/kreise_mit_center.csv')
#START
if False:
    ### All
    reader.readAllData()
else:
    ### Only one city
    city = reader.cities[1]
    reader.readData(city)
# END

s3Handler = S3.S3_Handler()
s3Handler.writeToAWS("airquality", reader.bucket)

## Test reading process

In [ ]:
import pandas as pd
from datetime import datetime
s3Handler = S3.S3_Handler()
listOfFile = s3Handler.listFromAWS("airquality", datetime.now())
fullData = []
count = 0
for pathItem in listOfFile:
    jsonItem = s3Handler.readFromAWS(pathItem)
    if jsonItem != False:
        fullData = fullData + jsonItem
        count += 1
if len(listOfFile)==count:
    print("All files have been read")
else:
    print("Not all files have been read")